In [1]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination 
from dotenv import load_dotenv
from autogen_agentchat.base import TaskResult
from autogen_agentchat.ui import Console
import os


load_dotenv()


True

In [2]:
async def team_Config(job_position="Software Engineer"):

    model_client = OpenAIChatCompletionClient(model="gpt-4o",api_key=os.getenv("OPENAI_API_KEY"))

    # Defining our Agent
    # 1. Interviewer Agent
    # 2. Interviewee Agent
    # 3. Career coach Agent
    job_position = "Data Science Manager"

    interviewer = AssistantAgent(
        name="Interviewer",
        model_client=model_client,
        description=f"An AI agent that conducts interviews for a {job_position} position.",
        system_message=f'''
        You are a professional interviewer for a {job_position} position.
        Ask one clear question at a time and Wait for user to respond. 
        Your job is to continue and ask questions, don't pay any attention to interview coach's response. 
        Make sure to ask question based on Candidate's answer and your expertise in the field.
        Ask 3 questions in total covering technical skills and experience, problem-solving abilities, and cultural fit.
        After asking 3 questions, say 'TERMINATE' at the end of the interview.
        Make question under 50 words.
    '''
    )

    candidate = UserProxyAgent(
        name = "candidate",
        description=f"An agent that simulates a candidate for a {job_position} position.",
        input_func=input
    )


    # 3. Interview Coach Agent
    interview_coach = AssistantAgent(
        name="Interview_Coach",
        model_client=model_client,
        description=f"An AI agent that provides feedback and advice to candidates for a {job_position} position to better prepare for interview",
        system_message=f'''
        You are a interview coach specializing in preparing candidates for {job_position} interviews.
        Provide constructive feedback on the candidate's responses and suggest improvements.
        After the interview, summarize the candidate's performance and provide actionable advice.
        Give a final rating to the candidate out of 0-10 with 10 being the best.
        Make your feedback under 100 words.
    '''
    )

    terminate_condition = TextMentionTermination(text="TERMINATE")

    team = RoundRobinGroupChat(
        participants=[interviewer, candidate, interview_coach],
        termination_condition=terminate_condition,
        max_turns=20
    )
    return team


async def interview(team):
# Running the interview
    async for message in team.run_stream(task='Start the interview with the first question ?'):
        if isinstance(message,TaskResult):
            message = f'Interview completed with result: {message.stop_reason}'
            yield message
        else:
            message = f'{message.source}: {message.content}'
            yield message


In [3]:
import asyncio

job_position = "Data Science Manager"
team = await team_Config(job_position)
from autogen_agentchat.ui import Console
await Console(team.run_stream())


---------- TextMessage (Interviewer) ----------
What technical skills and tools do you consider essential for a Data Science Manager, and how have you applied them in your previous roles?
---------- TextMessage (candidate) ----------
I think SQL , Python , Problem Solving , Advanced Machine Learning knowledge , Statistics and MLOps are required skills for a data science manager. I have applied them in my previous roles in an ecommerce company where I have built end to end recommendation systems.
---------- TextMessage (Interview_Coach) ----------
Your mention of technical skills like SQL, Python, machine learning, and MLOps shows you have a solid foundation. However, a more detailed explanation of how you utilized these skills, particularly in managing teams and projects, would have strengthened your response. Consider discussing specific tools (e.g., TensorFlow, Docker) and any frameworks for collaboration or project management (e.g., Agile, Jira) to showcase your managerial capabilit

TaskResult(messages=[TextMessage(id='63e99b5f-999f-49a5-b664-d4383c8629bc', source='Interviewer', models_usage=RequestUsage(prompt_tokens=126, completion_tokens=26), metadata={}, created_at=datetime.datetime(2025, 8, 9, 2, 54, 33, 670175, tzinfo=datetime.timezone.utc), content='What technical skills and tools do you consider essential for a Data Science Manager, and how have you applied them in your previous roles?', type='TextMessage'), UserInputRequestedEvent(id='00fd2556-874b-4996-b6af-9f935609f8f1', source='candidate', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 9, 2, 54, 33, 674526, tzinfo=datetime.timezone.utc), request_id='097cc23e-5cd4-4588-8754-5a1f7749bbed', content='', type='UserInputRequestedEvent'), TextMessage(id='43e3abe7-59aa-465c-aa98-e6a66db7b7e1', source='candidate', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 9, 2, 55, 55, 533930, tzinfo=datetime.timezone.utc), content='I think SQL , Python , Problem Solving , Adva